In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [4]:
hps = utils.get_hparams_from_file("./configs/genshin_base_ms.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint('./checkpoints/G_34000.pth', net_g, None)
import soundfile as sf
text = "你就是大名鼎鼎的旅行者吧" #@param {type: 'string'}
length_scale = 1.0 #@param {type:"slider", min:0.1, max:3, step:0.05}
filename = 'test' #@param {type: "string"}
audio_path = f'./output/{filename}.wav'
stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([0]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
sf.write(audio_path,audio,samplerate=hps.data.sampling_rate)

In [18]:
for param_tensor in net_g.state_dict():
    #打印 key value字典
    print(param_tensor,'\t',net_g.state_dict()[param_tensor].size())

enc_p.emb.weight 	 torch.Size([380, 192])
enc_p.encoder.attn_layers.0.emb_rel_k 	 torch.Size([1, 9, 96])
enc_p.encoder.attn_layers.0.emb_rel_v 	 torch.Size([1, 9, 96])
enc_p.encoder.attn_layers.0.conv_q.weight 	 torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_q.bias 	 torch.Size([192])
enc_p.encoder.attn_layers.0.conv_k.weight 	 torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_k.bias 	 torch.Size([192])
enc_p.encoder.attn_layers.0.conv_v.weight 	 torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_v.bias 	 torch.Size([192])
enc_p.encoder.attn_layers.0.conv_o.weight 	 torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_o.bias 	 torch.Size([192])
enc_p.encoder.attn_layers.1.emb_rel_k 	 torch.Size([1, 9, 96])
enc_p.encoder.attn_layers.1.emb_rel_v 	 torch.Size([1, 9, 96])
enc_p.encoder.attn_layers.1.conv_q.weight 	 torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.1.conv_q.bias 	 torch.Size([192])
enc_p.encoder.attn_layers.1.conv_k.weight 	 torch.Size([

In [33]:
import torch as th
emb_g = nn.Embedding(4, 4)
print(emb_g.weight)

Parameter containing:
tensor([[-1.4650, -1.2804,  0.0268,  1.4988],
        [-0.7531,  0.3802, -0.0263, -0.2750],
        [-1.0192, -0.5266,  0.0216,  0.7424],
        [ 1.6118, -1.4766, -0.1870, -0.2322]], requires_grad=True)


In [35]:
emb_g(th.LongTensor([0, 2])).unsqueeze(-1)

tensor([[[-1.4650],
         [-1.2804],
         [ 0.0268],
         [ 1.4988]],

        [[-1.0192],
         [-0.5266],
         [ 0.0216],
         [ 0.7424]]], grad_fn=<UnsqueezeBackward0>)